<a href="https://colab.research.google.com/github/rammiahj/GitTutorial/blob/master/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image, ImageDraw
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import torch.optim as optim
import torchvision
import glob


In [ ]:
#Dataloader

#Path for training and testing directory
train_img='/content/drive/MyDrive/ASLO/training'
test_img='/content/drive/MyDrive/ASLO/testing'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_img,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_img,transform=transformer),
    batch_size=32, shuffle=True
)
print(f"Train Classes : {len(train_img)}")
print(f"Test Classes : {len(test_img)}")

Train Classes : 36
Test Classes : 35


In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_img+'/**/*.tif'))
test_count=len(glob.glob(test_img+'/**/*.tif'))

print(train_count,test_count)

3300 3310


In [ ]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#print device
print('Using device : ',device)

Using device :  cpu


In [ ]:
!wget https://download.pytorch.org/models/resnet50-0676ba61.pth 
model = models.resnet50()
model.load_state_dict(torch.load('resnet50-0676ba61.pth'))

for param in model.parameters(): #Freezing the model
  param.requires_grad = False

model.fc = torch.nn.Linear(in_features=2048, out_features=22, bias=True)
#model.fc.weight.requires_grad=True

model=model.to(device)

--2021-06-21 09:39:01--  https://download.pytorch.org/models/resnet50-0676ba61.pth
Resolving download.pytorch.org (download.pytorch.org)... 13.226.50.98, 13.226.50.24, 13.226.50.77, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.50.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102530333 (98M) [application/x-www-form-urlencoded]
Saving to: ‘resnet50-0676ba61.pth’

resnet50-0676ba61.p 100%[===================>]  97.78M   119MB/s    in 0.8s    

2021-06-21 09:39:02 (119 MB/s) - ‘resnet50-0676ba61.pth’ saved [102530333/102530333]



In [ ]:
#Optimizer and Loss function
criterian = nn.CrossEntropyLoss() #CrossEntropyLoss for classification problem
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) #lr and weight_decay are hyper parameters 

In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(10):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=criterian(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

    # Evaluation on testing dataset
    #with torch.no_grad():
    model.eval()  
    test_accuracy=0.0

    for i, (images,labels) in enumerate(test_loader):
      if torch.cuda.is_available():
          images=Variable(images.cpu())
          labels=Variable(labels.cpu())
            
      outputs=model(images)
      _,prediction=torch.max(outputs.data,1)
      test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

print("Finished Training and Testing")

Epoch: 0 Train Loss: tensor(0.6297) Train Accuracy: 0.8375757575757575 Test Accuracy: 0.8429003021148036
Epoch: 1 Train Loss: tensor(0.4986) Train Accuracy: 0.8648484848484849 Test Accuracy: 0.8465256797583082
Epoch: 2 Train Loss: tensor(0.4074) Train Accuracy: 0.8860606060606061 Test Accuracy: 0.8625377643504532
Epoch: 3 Train Loss: tensor(0.3440) Train Accuracy: 0.9121212121212121 Test Accuracy: 0.8622356495468277
Epoch: 4 Train Loss: tensor(0.3222) Train Accuracy: 0.9087878787878788 Test Accuracy: 0.8628398791540786
Epoch: 5 Train Loss: tensor(0.2775) Train Accuracy: 0.9248484848484848 Test Accuracy: 0.86797583081571
Epoch: 6 Train Loss: tensor(0.2715) Train Accuracy: 0.926060606060606 Test Accuracy: 0.8649546827794562
Epoch: 7 Train Loss: tensor(0.2517) Train Accuracy: 0.9293939393939394 Test Accuracy: 0.8558912386706948
Epoch: 8 Train Loss: tensor(0.2457) Train Accuracy: 0.9336363636363636 Test Accuracy: 0.8694864048338369
Epoch: 9 Train Loss: tensor(0.2077) Train Accuracy: 0.9424